In [5]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import pyautogui
import pyperclip
from pathlib import Path
from datetime import datetime
import warnings

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
from tkcalendar import DateEntry

def aniver():
    
    pyautogui.alert("""Para pegar a Listagem de aniversariantes no ASA:
    
    1- Prontuario 
    2- Listagem 
    3- Listagem de pacientes 
    4- Filtrar por data de hoje
    5- Salvar como xlsx (excel)""")

    escolher_arquivo = askopenfilename(title='Favor, importar listagem dos Aniversariantes em excel')
    niver = pd.read_excel(escolher_arquivo) 
    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)

    novo_header = niver.iloc[3]
    novo_header = list(novo_header)
    niver.columns = novo_header

    niver = niver.drop([0,1,2,3,4])

    niver = niver.reset_index(drop=True)

    for i,item in enumerate(niver['Código']):
        if (item == 'Código') or (item == 'Tecnoarte Informática') or (item == '<128>') or (item == 'Gerson Schorr Clinica de Alergia e Dermatologia'): 
            niver = niver.drop(i)

    niver = niver.reset_index(drop=True)

    niver['Código'] = niver['Código'].astype(str)

    for i,item in enumerate(niver['Código']):
        if 'Data Selecionada' in item: 
            niver = niver.drop(i)

    niver = niver.reset_index(drop=True)

    for i,item in enumerate(niver['Paciente']):
        niver['Paciente'] = niver['Paciente'].str.strip()
        if item == 'Endereço': 
            niver = niver.drop(i)

    niver = niver.reset_index(drop=True)

    niver = niver[['Código','Paciente']]

    for i,item in enumerate(niver['Código']):
        if item != 'nan':
            niver = niver.drop(i + 1)
            niver = niver.drop(i + 2)
            niver = niver.drop(i + 3)

    niver = niver.reset_index(drop=True)

    niver = niver.dropna(subset=['Paciente'])

    niver = niver.reset_index(drop=True)

    for i,item in enumerate(niver['Código']):
        if item != 'nan':
            if niver.loc[i + 1, 'Código'] != 'nan':
                niver = niver.drop(i)

    niver = niver.reset_index(drop=True)            

    niver['Paciente'] = niver['Paciente'].str.replace('-', '',regex=True)
    niver['Paciente'] = niver['Paciente'].str.replace('(', '',regex=True)
    niver['Paciente'] = niver['Paciente'].str.replace(')', '',regex=True)
    niver['Paciente'] = niver['Paciente'].str.replace('/', '',regex=True)
    niver['Paciente'] = niver['Paciente'].str.replace('*', '',regex=True)
    niver['Paciente'] = niver['Paciente'].astype(str)

    for i,item in enumerate(niver['Paciente']):
        if item[0] == '0':
            niver['Paciente'] = niver['Paciente'].str.lstrip('0')

    for i,item in enumerate(niver['Código']):
        if item == 'nan':
            celular = niver.loc[i, 'Paciente']
            celular = celular.replace(' ','')
            if len(celular) != 11:
                niver = niver.drop(i)

    niver = niver.reset_index(drop=True) 

    lista_celulares = []
    lista_nomes = []
    lista_celulares_duplos = []

    for i,item in enumerate(niver['Paciente']):
        if item.isnumeric():
            lista_celulares.append(item)
            nome = niver.loc[i - 1, 'Paciente']
            lista_nomes.append(nome)

    for i, item in enumerate(lista_nomes):
        if item.isnumeric():
            lista_celulares_duplos.append(item)
            lista_nomes.pop(i)

    for i, celular in enumerate(lista_celulares):
        for celular_duplo in lista_celulares_duplos:
            if celular == celular_duplo:
                lista_celulares.pop(i)

    if len(lista_celulares) == len(lista_nomes):
        lista_tuplas_nome_celular = list(zip(lista_nomes, lista_celulares))

        aniversarios = pd.DataFrame(lista_tuplas_nome_celular, columns=['Nome', 'Celular'])

        for i, nome in enumerate(aniversarios['Nome']):
            aniversarios.loc[i , 'Nome'] = nome.title()

#         display(aniversarios.tail(50))

        for i, celular in enumerate(aniversarios['Celular']):

            hoje = datetime.today()
            hoje = hoje.strftime('%d/%m/%Y')

            nome = aniversarios.loc[i, 'Nome']
            
            print(nome)

            texto = f"""Boa tarde {nome}, falamos da Clínica Gerson Schorr de Alergia e Dermatologia 

            Hoje, dia {hoje} viemos com enorme prazer lhe desejar um feliz aniversário! 

            Que esse novo ciclo te traga muita prosperidade, amor e principalmente saúde.

            Parabéns e muitas felicidades!"""
            
            
            time.sleep(3)
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(3)

            try:
                # Adicionar Contato
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
                navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
                celular55 = '55' + celular             
                for i, carac_cel in enumerate(celular):                 
                    navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

                # Salvar Contato
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
                time.sleep(3)

                # Se o paciente não tiver contato salvo no celular da clinica
                try:
                    
                # Clicar na seta de mandar mensagem
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Se o paciente já tiver contato salvo no celular da clinica
                except:
                    time.sleep(2)
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(3)
                    navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                    time.sleep(2)
                    navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                    time.sleep(5)

                    # Pegar lista de contatos com nome parecido
                    lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                    tamanho_lista = len(lista_contatos_comparar)

                    for x in range(tamanho_lista):
                        if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                            navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                    time.sleep(5)

                    # Clicar na seta de mandar mensagem
                    try: 
                        navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                        time.sleep(3)
                        navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                        time.sleep(1)
                        navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                        navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                    # Caso já tenha uma conversa aberta com o paciente
                    except:
                        navegador.get('https://chat.sonax.net.br/#/app/chat')
                        time.sleep(2)

            except: # Caso aconteça algum erro no salvamento do contato
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(2)

        pyautogui.alert('Repescagem GSC Finalizada!')
        navegador.quit()
    
    else:
        print('Erro na Listagem de aniversariantes')

def repescagem_gsc():
    pyautogui.alert("""Para pegar o relatório de repescagem GSC no ASA:
    
    1- Agenda 
    2- Relatórios - Relatórios operacionais 
    3- Pacientes Faltosos da agenda médica 
    4- Selecionar "repesc gsc hoje"
    5- Salvar como xlsx (excel)""")
    
    escolher_arquivo = askopenfilename(title='Favor, importar Repescagem GSC em excel')
    repesc = pd.read_excel(escolher_arquivo) 
    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)
    
    # Tratando Planilha 
    
    repesc = repesc[['Unnamed: 1','Unnamed: 3','Unnamed: 4','Unnamed: 9']]
    repesc.columns = ['nome','data 2','data','celular']
    repesc = repesc.dropna(thresh=3)
    repesc = repesc[repesc['celular'] != 'Telefone']
    repesc = repesc.reset_index(drop=True)
    repesc['data'] = repesc['data'].fillna('x')

    # Unindo colunas de data
    lista_novas_datas = []

    for i, linha_data in enumerate(repesc['data']):
        if linha_data == 'x':
            linha_data = repesc.loc[i, 'data 2']
        lista_novas_datas.append(linha_data)

    repesc['data'] = lista_novas_datas

    repesc = repesc[['nome','data','celular']]

    # Mudando formatação da data
    lista_format_datas = []

    for data in repesc['data']:
        ano = data[6:10]
        mes = data[3:5]
        dia = data[0:2]
        hora = data[11:16]
        data = dia + '/' + mes + '/' + ano + ' ás ' + hora
        lista_format_datas.append(data)

    repesc['data'] = lista_format_datas

    # Tratando coluna dos celulares
    repesc['celular'] = repesc['celular'].astype(str)

    repesc['celular'] = repesc['celular'].str.replace('-', '',regex=True)
    repesc['celular'] = repesc['celular'].str.replace('(', '',regex=True)
    repesc['celular'] = repesc['celular'].str.replace(')', '',regex=True)
    repesc['celular'] = repesc['celular'].str.replace(' ', '',regex=True)

    for i, celular in enumerate(repesc['celular']):
        if len(celular) != 11:
            repesc = repesc.drop(i)

    repesc = repesc.reset_index(drop=True)

    for i, celular in enumerate(repesc['celular']):
        if celular[0] == '0':
            repesc = repesc.drop(i)

    repesc = repesc.reset_index(drop=True)
    repesc['celular'] = repesc['celular'].astype(str)

    # Tratando coluna de nome dos pacientes
    lista_novos_nomes = []

    for i, nome_paciente in enumerate(repesc['nome']):
        nome_paciente = nome_paciente.title()
        lista_novos_nomes.append(nome_paciente)

    repesc['nome'] = lista_novos_nomes
    
    repesc['celular'] = repesc['celular'].astype(str)
    
    # -------------------------------------------------------------------------
    
    for i, celular in enumerate(repesc['celular']):
        nome = repesc.loc[i, 'nome']
        data_marcada = repesc.loc[i, 'data']
        print('Paciente {}'.format(nome))

        time.sleep(3)
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(3)

        texto = f'Olá {nome}, falamos da Clinica Gerson Schorr de Alergia e Dermatologia.\n\nO sr(a) havia agendado uma consulta conosco dia {data_marcada} e não pode comparecer.\n\nGostaríamos de saber qual seria a disponibilidade do sr(a) para estar remarcando a sua consulta?\n\nAguardamos o seu retorno,obrigada.'
        
        try:
            # Adicionar Contato
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
            navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
            celular55 = '55' + celular             
            for i, carac_cel in enumerate(celular):                 
                navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

            # Salvar Contato
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            time.sleep(3)

            # Se o paciente não tiver contato salvo no celular da clinica
            try:
            # Clicar na seta de mandar mensagem
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Se o paciente já tiver contato salvo no celular da clinica
            except:
                time.sleep(2)
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(3)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                time.sleep(2)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                time.sleep(5)

                # Pegar lista de contatos com nome parecido
                lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                tamanho_lista = len(lista_contatos_comparar)

                for x in range(tamanho_lista):
                    if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                        navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                time.sleep(5)

                # Clicar na seta de mandar mensagem
                try: 
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Caso já tenha uma conversa aberta com o paciente
                except:
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(2)

        except: # Caso aconteça algum erro no salvamento do contato
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(2)

    pyautogui.alert('Repescagem GSC Finalizada!')
    navegador.quit()

def repescagem_cms():
    pyautogui.alert("""Para pegar o relatório de repescagem CMS no ASA:
    
    1- Agenda 
    2- Relatórios - Relatórios operacionais 
    3- Pacientes Faltosos da agenda médica 
    4- Selecionar "repesc cms hoje"
    5- Salvar como xlsx (excel)""")
        
    escolher_arquivo = askopenfilename(title='Favor, importar Repescagem CMS em excel')
    repesc = pd.read_excel(escolher_arquivo) 
    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)
    
    # Tratando Planilha 
    
    repesc = repesc[['Unnamed: 1','Unnamed: 3','Unnamed: 4','Unnamed: 9']]
    repesc.columns = ['nome','data 2','data','celular']
    repesc = repesc.dropna(thresh=3)
    repesc = repesc[repesc['celular'] != 'Telefone']
    repesc = repesc.reset_index(drop=True)
    repesc['data'] = repesc['data'].fillna('x')

    # Unindo colunas de data
    lista_novas_datas = []

    for i, linha_data in enumerate(repesc['data']):
        if linha_data == 'x':
            linha_data = repesc.loc[i, 'data 2']
        lista_novas_datas.append(linha_data)

    repesc['data'] = lista_novas_datas

    repesc = repesc[['nome','data','celular']]

    # Mudando formatação da data
    lista_format_datas = []

    for data in repesc['data']:
        ano = data[6:10]
        mes = data[3:5]
        dia = data[0:2]
        hora = data[11:16]
        data = dia + '/' + mes + '/' + ano + ' ás ' + hora
        lista_format_datas.append(data)

    repesc['data'] = lista_format_datas

    # Tratando coluna dos celulares
    repesc['celular'] = repesc['celular'].astype(str)

    repesc['celular'] = repesc['celular'].str.replace('-', '',regex=True)
    repesc['celular'] = repesc['celular'].str.replace('(', '',regex=True)
    repesc['celular'] = repesc['celular'].str.replace(')', '',regex=True)
    repesc['celular'] = repesc['celular'].str.replace(' ', '',regex=True)

    for i, celular in enumerate(repesc['celular']):
        if len(celular) != 11:
            repesc = repesc.drop(i)

    repesc = repesc.reset_index(drop=True)

    for i, celular in enumerate(repesc['celular']):
        if celular[0] == '0':
            repesc = repesc.drop(i)

    repesc = repesc.reset_index(drop=True)
    repesc['celular'] = repesc['celular'].astype(str)

    # Tratando coluna de nome dos pacientes
    lista_novos_nomes = []

    for i, nome_paciente in enumerate(repesc['nome']):
        nome_paciente = nome_paciente.title()
        lista_novos_nomes.append(nome_paciente)

    repesc['nome'] = lista_novos_nomes
    
    repesc['celular'] = repesc['celular'].astype(str)
    
    # -------------------------------------------------------------------------
    
    for i, celular in enumerate(repesc['celular']):
        nome = repesc.loc[i, 'nome']
        data_marcada = repesc.loc[i, 'data']
        print('Paciente {}'.format(nome))

        time.sleep(3)
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(3)

        texto = f'Olá {nome}, falamos do Centro Médico Schorr de Alergia e Dermatologia.\n\nO sr(a) havia agendado uma consulta conosco dia {data_marcada} e não pode comparecer.\n\nGostaríamos de saber qual seria a disponibilidade do sr(a) para estar remarcando a sua consulta?\n\nAguardamos o seu retorno,obrigada.'
        
        try:
            # Adicionar Contato
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
            navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
            celular55 = '55' + celular             
            for i, carac_cel in enumerate(celular):                 
                navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

            # Salvar Contato
            #navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            time.sleep(3)

            # Se o paciente não tiver contato salvo no celular da clinica
            try:
            # Clicar na seta de mandar mensagem
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Se o paciente já tiver contato salvo no celular da clinica
            except:
                time.sleep(2)
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(3)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                time.sleep(2)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                time.sleep(5)

                # Pegar lista de contatos com nome parecido
                lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                tamanho_lista = len(lista_contatos_comparar)

                for x in range(tamanho_lista):
                    if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                        navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                time.sleep(5)

                # Clicar na seta de mandar mensagem
                try: 
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Caso já tenha uma conversa aberta com o paciente
                except:
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(2)

        except: # Caso aconteça algum erro no salvamento do contato
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(2)

    pyautogui.alert('Repescagem CMS Finalizada!')
    navegador.quit()

def confirmar_cms():
    pyautogui.alert("""Para pegar o relatório de Confirmação CMS no ASA:
    
    1- Agenda 
    2- Operações - Email/SMS 
    3- Filtrar Empresa para Centro Médico 
    4- Filtrar data do dia seguinte nos dois campos "atendido entre"
    5- Salvar como xlsx (excel)""")
    
    escolher_arquivo = askopenfilename(title='Favor, importar Confirmação CMS em excel')
    confirmacao_bruto = pd.read_excel(escolher_arquivo) 

    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)

        # Tratando Planilha Bruta --------------------------------------------------------------------------

    # Pegar Linha [2] da planilha
    novo_header = confirmacao_bruto.iloc[2]

    # Transformar essa linha [2] em lista
    novo_header = list(novo_header)

    # colocar essa lista como cabeçalho
    confirmacao_bruto.columns = novo_header

    # escolher colunas uteis da planilha com base no NOME delas
    confirmacao_bruto = confirmacao_bruto[['Código','Paciente','Data Marcada','Telefone 1','Telefone 2','Telefone 3']]
    confirmacao_bruto.columns = ['cod paciente', 'nome','data marcada', 'celular', 'celular 2', 'celular 3']

    confirmacao_bruto = confirmacao_bruto.dropna(subset=['nome']) 
    confirmacao_bruto = confirmacao_bruto[confirmacao_bruto['nome'] != 'Paciente']
    confirmacao_bruto = confirmacao_bruto.reset_index(drop=True)

    # Formatando colunas de celular
    def format_celular(nome_df, nome_coluna):
        nome_df[nome_coluna] = nome_df[nome_coluna].astype(str)
        
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace('-', '',regex=True)
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace('(', '',regex=True)
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace(')', '',regex=True)
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace(' ', '',regex=True)

        nome_df[nome_coluna] = nome_df[nome_coluna].astype(str)

        for i, celular in enumerate(nome_df[nome_coluna]):
            if len(celular) != 11:
                nome_df.loc[i, nome_coluna] = ' '

        for i, celular in enumerate(nome_df[nome_coluna]):
            if celular[0] == '0':
                nome_df.loc[i, nome_coluna] = ' '

    format_celular(confirmacao_bruto, 'celular')
    format_celular(confirmacao_bruto, 'celular 2')
    format_celular(confirmacao_bruto, 'celular 3')

    # Juntando Colunas de celular
    def junta_coluna(nome_df, coluna_vazia, coluna_celular):
        for i, celular in enumerate(nome_df[coluna_vazia]):
            if celular == ' ':
                nome_df.loc[i, coluna_vazia] = nome_df.loc[i, coluna_celular]

    junta_coluna(confirmacao_bruto, 'celular', 'celular 2')
    confirmacao_bruto = confirmacao_bruto[['cod paciente', 'nome','data marcada', 'celular', 'celular 3']]

    junta_coluna(confirmacao_bruto, 'celular', 'celular 3')
    confirmacao_bruto = confirmacao_bruto[['cod paciente', 'nome', 'data marcada', 'celular']]    

    for i, celular in enumerate(confirmacao_bruto['celular']):        
        if celular == ' ':  
            confirmacao_bruto = confirmacao_bruto.drop(i)

    confirmacao_bruto = confirmacao_bruto.reset_index(drop=True)

    # Editando texto da data
    lista_novas_datas = []

    for data in confirmacao_bruto['data marcada']:
        ano = data[6:10]
        mes = data[3:5]
        dia = data[0:2]
        hora = data[11:16]
        data = dia + '/' + mes + '/' + ano + ' ás ' + hora
        lista_novas_datas.append(data)

    confirmacao_bruto['data marcada'] = lista_novas_datas

    confirmacao_bruto['celular'] = confirmacao_bruto['celular'].astype(str)

#     display(confirmacao_bruto)

    # -----------------------------------------------------------------------------------------------------

    for i, celular in enumerate(confirmacao_bruto['celular']):
        nome = confirmacao_bruto.loc[i, 'nome']
        data_marcada = confirmacao_bruto.loc[i, 'data marcada']
        print('Paciente {}'.format(nome))

        time.sleep(3)
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(3)

        texto = f'Olá {nome}, Falamos do Centro Médico Schorr de Alergia e Dermatologia para confirmar o seu agendamento do dia {data_marcada}.\n\n*Avenida Nossa Senhora de Copacabana, 500 sala 711-712*\n\nEntre as ruas Hilário de Gouveia e Paula Freitas\n\nMetrô Siqueira Campos'
        
        try:
            # Adicionar Contato
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
            navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
            celular55 = '55' + celular             
            for i, carac_cel in enumerate(celular):                 
                navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

            # Salvar Contato
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            time.sleep(3)

            # Se o paciente não tiver contato salvo no celular da clinica
            try:
            # Clicar na seta de mandar mensagem
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Se o paciente já tiver contato salvo no celular da clinica
            except:
                time.sleep(2)
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(3)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                time.sleep(2)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                time.sleep(5)

                # Pegar lista de contatos com nome parecido
                lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                tamanho_lista = len(lista_contatos_comparar)

                for x in range(tamanho_lista):
                    if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                        navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                time.sleep(5)

                # Clicar na seta de mandar mensagem
                try: 
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Caso já tenha uma conversa aberta com o paciente
                except:
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(2)

        except: # Caso aconteça algum erro no salvamento do contato
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(2)

    pyautogui.alert('Confirmação CMS Finalizada!')
    navegador.quit()


def confirmar_gsc():
    pyautogui.alert("""Para pegar o relatório de Confirmação GSC no ASA:
    
    1- Agenda 
    2- Operações - Email/SMS 
    3- Filtrar Empresa para Gerson Schorr 
    4- Filtrar data do dia seguinte nos dois campos "atendido entre"
    5- Salvar como xlsx (excel)""")
    
    escolher_arquivo = askopenfilename(title='Favor, importar confirmação GSC em excel')
    confirmacao_bruto = pd.read_excel(escolher_arquivo) 

    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)

        # Tratando Planilha Bruta --------------------------------------------------------------------------

    # Pegar Linha [2] da planilha
    novo_header = confirmacao_bruto.iloc[2]

    # Transformar essa linha [2] em lista
    novo_header = list(novo_header)

    # colocar essa lista como cabeçalho
    confirmacao_bruto.columns = novo_header

    # escolher colunas uteis da planilha com base no NOME delas
    confirmacao_bruto = confirmacao_bruto[['Código','Paciente','Data Marcada','Telefone 1','Telefone 2','Telefone 3']]
    confirmacao_bruto.columns = ['cod paciente', 'nome','data marcada', 'celular', 'celular 2', 'celular 3']

    confirmacao_bruto = confirmacao_bruto.dropna(subset=['nome']) 
    confirmacao_bruto = confirmacao_bruto[confirmacao_bruto['nome'] != 'Paciente']
    confirmacao_bruto = confirmacao_bruto.reset_index(drop=True)

    # Formatando colunas de celular
    def format_celular(nome_df, nome_coluna):
        nome_df[nome_coluna] = nome_df[nome_coluna].astype(str)
        
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace('-', '',regex=True)
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace('(', '',regex=True)
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace(')', '',regex=True)
        nome_df[nome_coluna] = nome_df[nome_coluna].str.replace(' ', '',regex=True)

        nome_df[nome_coluna] = nome_df[nome_coluna].astype(str)

        for i, celular in enumerate(nome_df[nome_coluna]):
            if len(celular) != 11:
                nome_df.loc[i, nome_coluna] = ' '

        for i, celular in enumerate(nome_df[nome_coluna]):
            if celular[0] == '0':
                nome_df.loc[i, nome_coluna] = ' '

    format_celular(confirmacao_bruto, 'celular')
    format_celular(confirmacao_bruto, 'celular 2')
    format_celular(confirmacao_bruto, 'celular 3')

    # Juntando Colunas de celular
    def junta_coluna(nome_df, coluna_vazia, coluna_celular):
        for i, celular in enumerate(nome_df[coluna_vazia]):
            if celular == ' ':
                nome_df.loc[i, coluna_vazia] = nome_df.loc[i, coluna_celular]

    junta_coluna(confirmacao_bruto, 'celular', 'celular 2')
    confirmacao_bruto = confirmacao_bruto[['cod paciente', 'nome','data marcada', 'celular', 'celular 3']]

    junta_coluna(confirmacao_bruto, 'celular', 'celular 3')
    confirmacao_bruto = confirmacao_bruto[['cod paciente', 'nome', 'data marcada', 'celular']]    

    for i, celular in enumerate(confirmacao_bruto['celular']):        
        if celular == ' ':  
            confirmacao_bruto = confirmacao_bruto.drop(i)

    confirmacao_bruto = confirmacao_bruto.reset_index(drop=True)

    # Editando texto da data
    lista_novas_datas = []

    for data in confirmacao_bruto['data marcada']:
        ano = data[6:10]
        mes = data[3:5]
        dia = data[0:2]
        hora = data[11:16]
        data = dia + '/' + mes + '/' + ano + ' ás ' + hora
        lista_novas_datas.append(data)

    confirmacao_bruto['data marcada'] = lista_novas_datas

    confirmacao_bruto['celular'] = confirmacao_bruto['celular'].astype(str)

#     display(confirmacao_bruto)

    # -----------------------------------------------------------------------------------------------------

    for i, celular in enumerate(confirmacao_bruto['celular']):
        nome = confirmacao_bruto.loc[i, 'nome']
        data_marcada = confirmacao_bruto.loc[i, 'data marcada']
        print('Paciente {}'.format(nome))

        time.sleep(3)
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(3)

        texto = f"Olá {nome}, Falamos da Clínica Gerson Schorr de Alergia e Dermatologia para confirmar o seu agendamento do dia {data_marcada}.\n\n*Av. Rio Branco, 257 sala 710-714*\n\nEsquina com a Rua Santa Luzia\n\nMetrô - Cinelâdia, saída E."

        try:
            # Adicionar Contato
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
            navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
            celular55 = '55' + celular
            for i, carac_cel in enumerate(celular):
                navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])
            
            # Salvar Contato
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            time.sleep(3)

            # Se o paciente não tiver contato salvo no celular da clinica
            try:
            # Clicar na seta de mandar mensagem
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Se o paciente já tiver contato salvo no celular da clinica
            except:
                time.sleep(2)
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(3)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                time.sleep(2)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                time.sleep(5)

                # Pegar lista de contatos com nome parecido
                lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                tamanho_lista = len(lista_contatos_comparar)

                for x in range(tamanho_lista):
                    if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                        navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                time.sleep(5)

                # Clicar na seta de mandar mensagem
                try: 
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Caso já tenha uma conversa aberta com o paciente
                except:
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(2)

        except: # Caso aconteça algum erro no salvamento do contato
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(2)

    pyautogui.alert('Confirmação GSC Finalizada!')
    navegador.quit()
    
def repesc_vaci_gsc(): 
    pyautogui.alert("""Para pegar o relatório de repescagem de vacina GSC no ASA:
    
    1- Agenda 
    2- Relatórios - Relatórios operacionais 
    3- Pacientes Faltosos da agenda médica 
    4- Selecionar "repesc gsc hoje"
    5- Na aba Médico selecionar apenas "Imunoterapia-Vacina"
    6- Na aba Período selecionar o mes de referência
    5- Salvar como xlsx (excel)""")
        
    escolher_arquivo = askopenfilename(title='Favor, importar repescagem de vacinas mensal da GSC em excel')
    faltosos_vacina = pd.read_excel(escolher_arquivo) 
    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(4)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)

    # Tratando Planilha 

    faltosos_vacina = faltosos_vacina.drop([0,1,2,3])
    faltosos_vacina = faltosos_vacina[['Unnamed: 1','Unnamed: 4','Unnamed: 9']]

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    faltosos_vacina = faltosos_vacina.dropna()

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    nomes_paciente = []

    for paciente in faltosos_vacina['Unnamed: 1']:
        paciente = paciente.title()
        nomes_paciente.append(paciente)

    faltosos_vacina['Unnamed: 1'] = nomes_paciente

    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].astype(str)

    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace('-', '',regex=True)
    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace('(', '',regex=True)
    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace(')', '',regex=True)
    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace(' ', '',regex=True)

    for i, celular in enumerate(faltosos_vacina['Unnamed: 9']):
        if len(celular) != 11:
            faltosos_vacina = faltosos_vacina.drop(i)

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    for i, celular in enumerate(faltosos_vacina['Unnamed: 9']):
        if celular[0] == '0':
            faltosos_vacina = faltosos_vacina.drop(i)

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].astype(str)

    nova_data = []

    for data in faltosos_vacina['Unnamed: 4']:
        data = data[:10]
        nova_data.append(data)

    faltosos_vacina['Unnamed: 4'] = nova_data

    faltosos_vacina.columns = ['nome','data','celular']

#     display(faltosos_vacina)

    for i, celular in enumerate(faltosos_vacina['celular']):
        nome = faltosos_vacina.loc[i, 'nome']
        data_marcada = faltosos_vacina.loc[i, 'data']
        print('Paciente {}'.format(nome))

        time.sleep(3)
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(3)

        texto = f"""
        Olá {nome}, falamos da Clinica Gerson Schorr de Alergia e Dermatologia.\n
        O sr(a) havia agendado uma consulta com vacina referente ao tratamento de imunoterapia em {data_marcada} e não pôde comparecer.\n
        Gostaríamos de saber qual seria a disponibilidade do sr(a) para estar remarcando e dar continuidade no tratamento?\n
        Aguardamos o seu retorno, obrigada.
        """
            
        try:
            # Adicionar Contato
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
            navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
            celular55 = '55' + celular             
            for i, carac_cel in enumerate(celular):                 
                navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

            # Salvar Contato
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            time.sleep(3)

            # Se o paciente não tiver contato salvo no celular da clinica
            try:
            # Clicar na seta de mandar mensagem
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Se o paciente já tiver contato salvo no celular da clinica
            except:
                time.sleep(2)
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(3)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                time.sleep(2)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                time.sleep(5)

                # Pegar lista de contatos com nome parecido
                lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                tamanho_lista = len(lista_contatos_comparar)

                for x in range(tamanho_lista):
                    if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                        navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                time.sleep(5)

                # Clicar na seta de mandar mensagem
                try: 
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Caso já tenha uma conversa aberta com o paciente
                except:
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(2)

        except: # Caso aconteça algum erro no salvamento do contato
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(2)

    pyautogui.alert('Repescagem de vacinas GSC Finalizada!')
    navegador.quit()

def repesc_vaci_cms(): 
    pyautogui.alert("""Para pegar o relatório de repescagem de vacina CMS no ASA:
    
    1- Agenda 
    2- Relatórios - Relatórios operacionais 
    3- Pacientes Faltosos da agenda médica 
    4- Selecionar "repesc cms hoje"
    5- Na aba Médico selecionar apenas "Equipe Imuno-Vacina Copa"
    6- Na aba Período selecionar o mes de referência
    5- Salvar como xlsx (excel)""")
        
    escolher_arquivo = askopenfilename(title='Favor, importar repescagem de vacinas mensal da CMS em excel')
    faltosos_vacina = pd.read_excel(escolher_arquivo) 
    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(4)
    navegador.find_element(By.XPATH,'//*[@id="user"]').send_keys('marketing@gersonschorr.com.br')
    navegador.find_element(By.XPATH,'//*[@id="pass"]').send_keys('copacabana500')
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[2]/div/div/div/div[1]/div/div[1]/div[3]/button').click()
    time.sleep(3)

    # Tratando Planilha 

    faltosos_vacina = faltosos_vacina.drop([0,1,2,3])
    faltosos_vacina = faltosos_vacina[['Unnamed: 1','Unnamed: 4','Unnamed: 9']]

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    faltosos_vacina = faltosos_vacina.dropna()

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    nomes_paciente = []

    for paciente in faltosos_vacina['Unnamed: 1']:
        paciente = paciente.title()
        nomes_paciente.append(paciente)

    faltosos_vacina['Unnamed: 1'] = nomes_paciente

    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].astype(str)

    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace('-', '',regex=True)
    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace('(', '',regex=True)
    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace(')', '',regex=True)
    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].str.replace(' ', '',regex=True)

    for i, celular in enumerate(faltosos_vacina['Unnamed: 9']):
        if len(celular) != 11:
            faltosos_vacina = faltosos_vacina.drop(i)

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    for i, celular in enumerate(faltosos_vacina['Unnamed: 9']):
        if celular[0] == '0':
            faltosos_vacina = faltosos_vacina.drop(i)

    faltosos_vacina = faltosos_vacina.reset_index(drop=True)

    faltosos_vacina['Unnamed: 9'] = faltosos_vacina['Unnamed: 9'].astype(str)

    nova_data = []

    for data in faltosos_vacina['Unnamed: 4']:
        data = data[:10]
        nova_data.append(data)

    faltosos_vacina['Unnamed: 4'] = nova_data

    faltosos_vacina.columns = ['nome','data','celular']

#     display(faltosos_vacina)

    for i, celular in enumerate(faltosos_vacina['celular']):
        nome = faltosos_vacina.loc[i, 'nome']
        data_marcada = faltosos_vacina.loc[i, 'data']
        print('Paciente {}'.format(nome))

        time.sleep(3)
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(3)

        texto = f"""
        Olá {nome}, falamos do Centro Medico Schorr de Alergia e Dermatologia.\n
        O sr(a) havia agendado uma consulta com vacina referente ao tratamento de imunoterapia em {data_marcada} e não pôde comparecer.\n
        Gostaríamos de saber qual seria a disponibilidade do sr(a) para estar remarcando e dar continuidade no tratamento?\n
        Aguardamos o seu retorno, obrigada.
        """
            
        try:
            # Adicionar Contato
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
            navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
            celular55 = '55' + celular             
            for i, carac_cel in enumerate(celular):                 
                navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

            # Salvar Contato
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
            time.sleep(3)

            # Se o paciente não tiver contato salvo no celular da clinica
            try:
            # Clicar na seta de mandar mensagem
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Se o paciente já tiver contato salvo no celular da clinica
            except:
                time.sleep(2)
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(3)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
                time.sleep(2)
                navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
                time.sleep(5)

                # Pegar lista de contatos com nome parecido
                lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

                tamanho_lista = len(lista_contatos_comparar)

                for x in range(tamanho_lista):
                    if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                        navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
                time.sleep(5)

                # Clicar na seta de mandar mensagem
                try: 
                    navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                    time.sleep(3)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                    time.sleep(1)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                    navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto)
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

                # Caso já tenha uma conversa aberta com o paciente
                except:
                    navegador.get('https://chat.sonax.net.br/#/app/chat')
                    time.sleep(2)

        except: # Caso aconteça algum erro no salvamento do contato
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(2)

    pyautogui.alert('Repescagem de vacinas CMS Finalizada!')
    navegador.quit()


    
# --------------------------------------------------------------------------------------------------------------------------------------
    
def wpp_gsc():

    janela_gsc = tk.Toplevel()

    janela_gsc.title('Wpp GSC')
    
    janela_gsc.wm_iconbitmap('wpp_azul.ico')

    janela_gsc.geometry("%dx%d+%d+%d" % (235,250,570,280))

    label_gsc = tk.Label(janela_gsc, text='GSC :', bg='mediumorchid', fg='white', height=2, width=30)
    label_gsc.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

    botao_conf_gsc = tk.Button(janela_gsc, text='Confirmar Agenda GSC', command=confirmar_gsc, height=1, width=30)
    botao_conf_gsc.grid(row=1, column=0, pady=10)

    botao_repesc_gsc = tk.Button(janela_gsc, text='Repescar Agenda/Dia GSC', command=repescagem_gsc, height=1, width=30)
    botao_repesc_gsc.grid(row=2, column=0, pady=10)
    
    botao_repesc_gsc_vaci = tk.Button(janela_gsc, text='Repescar Vacinas/Mês GSC', command=repesc_vaci_gsc, height=1, width=30)
    botao_repesc_gsc_vaci.grid(row=3, column=0, pady=10)
    
    botao_voltar = tk.Button(janela_gsc, text='VOLTAR', command=janela_gsc.destroy, height=1, width=10)
    botao_voltar.grid(row=4, column=0, columnspan=2, padx=2, pady=10)
    
    janela_gsc.mainloop()

    # ----------------------------------------------------------------------------------
    
def wpp_cms():
    
    janela_cms = tk.Toplevel()

    janela_cms.title('Wpp CMS')
    
    janela_cms.wm_iconbitmap('wpp_azul.ico')

    janela_cms.geometry("%dx%d+%d+%d" % (235,250,570,280))

    label_cms = tk.Label(janela_cms, text='CMS :', bg='skyblue', fg='black',height=2, width=30)
    label_cms.grid(row=0, column=0, columnspan=2 ,padx=10 ,pady=10)

    botao_conf_cms = tk.Button(janela_cms, text='Confirmar Agenda CMS', command=confirmar_cms, height=1, width=30)
    botao_conf_cms.grid(row=1, column=0, pady=10)
    
    botao_repesc_cms = tk.Button(janela_cms, text='Repescar Agenda/Dia CMS', command=repescagem_cms, height=1, width=30)
    botao_repesc_cms.grid(row=2, column=0, pady=10)
    
    botao_repesc_cms_vaci = tk.Button(janela_cms, text='Repescar Vacinas/Mês CMS', command=repesc_vaci_cms, height=1, width=30)
    botao_repesc_cms_vaci.grid(row=3, column=0, pady=10)

    botao_voltar = tk.Button(janela_cms, text='VOLTAR', command=janela_cms.destroy, height=1, width=10)
    botao_voltar.grid(row=4, column=0, columnspan=2, padx=2, pady=10)

    janela_cms.mainloop()
    
# --------------------------------------------------------------------------------------------------------------------------------------
       
janela = tk.Tk()

janela.wm_iconbitmap('wpp_azul.ico')

janela.title('Automação Sonax Whatsapp')

janela.geometry("%dx%d+%d+%d" % (376,130,500,300))

label = tk.Label(text='A CONFIRMAÇÃO ou REPESCAGEM é referente à qual empresa?', bg='white', fg='black',height=2, width=50, relief="ridge")
label.grid(row=0, column=0, columnspan=2 ,padx=10 ,pady=10)

botao_gsc = tk.Button(text='GSC', bg='mediumorchid', fg='white', command=wpp_gsc, height=2, width=22)
botao_gsc.grid(row=1, column=0, columnspan=1, padx=2, pady=10)

botao_cms = tk.Button(text='CMS', bg='skyblue', fg='black',command=wpp_cms, height=2, width=22)
botao_cms.grid(row=1, column=1, columnspan=1, padx=2, pady=10)

# botao_niver = tk.Button(text='ANIVERSARIOS', bg='yellow2', fg='black',command=aniver, height=2, width=22)
# botao_niver.grid(row=2, column=0, columnspan=1, padx=2, pady=5)

janela.mainloop()

Gerson Schorr Clinica de Alergia e Dermatologia                      E-Mail
Unnamed: 1                                                              SMS
Unnamed: 2                                                             Conf
Unnamed: 3                                                           Código
Unnamed: 4                                                         Paciente
Unnamed: 5                                                             Item
Unnamed: 6                                                           Médico
Unnamed: 7                                                     Data Marcada
Unnamed: 8                                                           E-mail
Unnamed: 9                                                       Telefone 1
Unnamed: 10                                                      Telefone 2
Unnamed: 11                                                      Telefone 3
Unnamed: 12                                           Data Ult. Confirmação
Unnamed: 13 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Ananete\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Ananete\AppData\Local\Temp\ipykernel_18688\516129394.py", line 705, in confirmar_cms
    navegador.get('https://chat.sonax.net.br/#/app/chat')
NameError: name 'navegador' is not defined
